In [ ]:
pip install tika

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32625 sha256=f24441b07aeca77aaf8ee345475af44497269c8444d802f20beef2f837984d79
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [ ]:
from tika import parser
import re
import joblib
import pandas as pd
import numpy as np

In [ ]:
#finalcode

#Creating Function : 
# Taking 2 paramaters the first one is the path of the pdf and the last one is the word we wanna search for in the pdf. 

def pdfMining(path,x):
    
    #ourpdf
    pdf = path
    #for handling similar name for same thing that different from lab test to other         
    dict_cbc_names = {'Haemoglobin' : 'HB' , 
                       'WBCs' : 'Leucocytic'}


    #Parsing the content of our pdf into a list of sentences
    rawText1 = parser.from_file(pdf)
    rawList1 = rawText1['content'].splitlines()
    #Iterating inside the list of sentences
    for i in rawList1:

        if re.search(x, i, re.IGNORECASE):
            if 'MCHC' in i:
                continue
            #Our chosen sentence
            
            #Converting our little sentence to a list and splitting it into words to finally extract the needed number.
            list_ = list(i.split())
            list_ = str(list_)
             
            #Appling re to find the number after a certain word
            #Result is the number we need (VERY IMPORTANT)
            try :           
              result = re.search('\d+\.{0,1}\d*', list_).group()
              result = float(result)
              if 'Thousands' in list_ : 
                    result = result * 1000       
            except :
              continue
            
  
        elif str(x) == 'Haemoglobin' :
            if 'MCHC' in i:
                continue                               
            if dict_cbc_names['Haemoglobin'] in i : 
              x = dict_cbc_names[str(x)]
              list_ = list(i.split())
              list_ = str(list_)
              
              #Appling re to find the number after a certain word
              #Result is the number we need (VERY IMPORTANT)
              try :           
                result = re.search('\d+\.{0,1}\d*', list_).group()
                result = float(result)
              except :
                continue


        elif str(x) == 'WBCs' :
            if 'MCHC' in i:
                continue                               
            if dict_cbc_names['WBCs'] in i : 
                x = dict_cbc_names[str(x)]
                list_ = list(i.split())
                list_ = str(list_)                
                #Appling re to find the number after a certain word
                #Result is the number we need (VERY IMPORTANT)
                try :           
                  result = re.search('\d+\.{0,1}\d*', list_).group()
                  result = float(result)
                  if 'Thousands' in list_ : 
                    result = result * 1000                     
                except :
                  continue

            elif  'WBC' in i:
                x = 'WBC'                                 
                list_ = list(i.split())
                list_ = str(list_)
                  
                  #Appling re to find the number after a certain word
                  #Result is the number we need (VERY IMPORTANT)
                try :           
                    result = re.search('\d+\.{0,1}\d*', list_).group()
                    result = float(result)
                    if 'Thousands' in list_ : 
                      result = result * 1000                     
                except :
                    continue   
    
    return result
        

In [ ]:
pdfMining('Sample/S2.pdf','WBCs')

2023-05-30 10:46:33,253 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2023-05-30 10:46:37,381 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2023-05-30 10:46:38,453 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2023-05-30 10:46:43,469 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


6500.0

In [ ]:
pdfMining('Sample/S3.pdf','WBCs')

5500.0

In [ ]:
pdfMining('Sample/S1.pdf','WBCs')

6000.0

In [ ]:
pdfMining('Sample/S2.pdf','Platelet') 

248000.0

In [ ]:
pdfMining('Sample/S1.pdf','Platelet') 

199000.0

In [ ]:
pdfMining('Sample/S3.pdf','Platelet') 

155500.0

In [ ]:
try : 

              model_acute_l = joblib.load('Model/RandomForestModel(AcuteL-m,f).h5')
              model_cml = joblib.load('Model/RandomForestModel(CML-m,f).h5')
              features = list(model_acute_l.feature_names_in_)  + list(model_cml.feature_names_in_)
              features.pop(3) # Delete Duplicated Gender_Male from Feature List
              features.pop(3) # Delete Duplicated Hemoglobin from Feature List
              features.pop(3) # Delete Duplicated Platelets from Feature List
              data_dic = {}
              data_dic[features[0]] = pdfMining('Sample/S2.pdf','Haemoglobin')
              data_dic[features[1]] = pdfMining('Sample/S2.pdf','WBCs')
              data_dic[features[2]] = pdfMining('Sample/S2.pdf','Platelet')
              data_dic[features[3]] = pdfMining('Sample/S2.pdf','Basophils') 
              data_dic[features[4]] = pdfMining('Sample/S2.pdf','Eosinophils') 
              data_dic[features[5]] = np.random.choice([0,1])
              custom_data = pd.DataFrame(data = [data_dic] )   
              print('-----'*10)
              print('Your Input :\n',custom_data)
              print('-----'*10)       
              pred_acute_l  = model_acute_l.predict(custom_data[list(model_acute_l.feature_names_in_)])[0]
              pred_cml  = model_cml.predict(custom_data[list(model_cml.feature_names_in_)])[0]
              if pred_cml == pred_acute_l :
                                          # As predication of two models is same  
                                          # Two models have predicted Good as Label
                                          print('Based on Your Complete Blood Count (CBC) :')
                                          print('------'*10)    
                                          print('\t There is No Probability of Acute Leukmia and Chronic Myelogenous Leukemia (CML) ')
                                          output = 'Good'
                                          
              elif pred_cml == 'CML' and pred_acute_l == 'Acute L' :
                                          # As predication of two models is different 
                                          # One model has predicted CML as Label  
                                          # Other model has  predicted Acute L as Label  
                                          print('Based on Your Complete Blood Count (CBC) :')
                                          print('-----'*10)    
                                          print(
                                              '''
                                            There is Probability of Acute Leukmia and Chronic Myelogenous Leukemia (CML) :
                                                  >> As Your ['Hemoglobin' , 'WBCs', 'Platelets', 'Basophilis','Eosinophilia'] may be not in normal ranges 
                                            '''
                                              )
                                          output = 'Acute  L or CML'
      
              elif pred_cml == 'CML' and pred_acute_l == 'Good' :
                                          # As predication of two models is different 
                                          # One model has predicted CML as Label  
                                          # Other model has  predicted Good as Label  
                                          print('Based on Your Complete Blood Count (CBC) :')
                                          print('-----'*10)    
                                          print(
                                              '''
                                            There is Probability of Chronic Myelogenous Leukemia (CML) :
                                                  >> As Your ['Hemoglobin', 'Platelets','Basophilis','Eosinophilia'] may be not in normal ranges 
                                            ''' 
                                            )
                                          output = 'CML'
                                          
              elif pred_cml == 'Good' and pred_acute_l == 'Acute L' :
                                          # As predication of two models is different 
                                          # One model has predicted Good as Label  
                                          # Other model has  predicted Acute L as Label  
                                          print('Based on Your Complete Blood Count (CBC) :')
                                          print('-----'*10)    
                                          print(
                                              '''
                                          There is Probability of Acute Leukmia : 
                                              >> As Your ['Hemoglobin' , 'WBCs', 'Platelets'] may be not in normal ranges 
                                            ''' 
                                          )
                                          output = 'Acute L'             
             
except : 

        print('Your uploaded PDF can\'t be detected')
        print('-----'*10)  
        model_acute_l = joblib.load('Model/RandomForestModel(AcuteL-m,f).h5')
        model_cml = joblib.load('Model/RandomForestModel(CML-m,f).h5')
        features = list(model_acute_l.feature_names_in_)  + list(model_cml.feature_names_in_)
        features.pop(3) # Delete Duplicated Gender_Male from Feature List
        features.pop(3) # Delete Duplicated Hemoglobin from Feature List
        features.pop(3) # Delete Duplicated Platelets from Feature List                            
        print('\t Enter Manually the Following Please : ',features[:-1])


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


--------------------------------------------------
Your Input :
    Hemoglobin    WBCs  Platelets  Basophilis  Eosinophilia  Gender_Male
0        13.4  6500.0   248000.0         0.6           1.7            1
--------------------------------------------------
Based on Your Complete Blood Count (CBC) :
------------------------------------------------------------
	 There is No Probability of Acute Leukmia and Chronic Myelogenous Leukemia (CML) 
